## Урок 7. Вебинар “Производная функции одной переменной”

### Найти производную функции:

---
1. $$y = \frac{1}{x} + \frac{2}{x^2} - \frac{5}{x^3} + \sqrt{x} - \sqrt[3]{x} + \frac{3}{\sqrt{x}}$$

$$
y' = \left(\frac{1}{x} + \frac{2}{x^2} - \frac{5}{x^3} + \sqrt{x} - \sqrt[3]{x} + \frac{3}{\sqrt{x}}\right)' = - \frac{1}{x^2} - \frac{4}{x^3} + \frac{15}{x^4} + \frac{1}{2\sqrt{x}} - \frac{1}{3\sqrt[3]{x^2}} - \frac{3}{2\sqrt{x^3}}
$$

---
2. $$y = x \cdot \sqrt{1 + x^2}$$

$$
y' = \left(x \cdot \sqrt{1 + x^2}\right)' = (x)' \cdot \sqrt{1 + x^2} + \left(\sqrt{1 + x^2}\right)' \cdot x = 1 \cdot \sqrt{1 + x^2} + \frac{1}{2\sqrt{1 + x^2}} \cdot 2x \cdot x =
$$

$$
= \frac{\sqrt{1 + x^2} \cdot \sqrt{1 + x^2}}{\sqrt{1 + x^2}} + \frac{x^2}{\sqrt{1 + x^2}} = \frac{1 + x^2 + x^2}{\sqrt{1 + x^2}} = \frac{1 + 2x^2}{\sqrt{1 + x^2}}
$$

Вариант более простого решения:

$$
y' = \left(x \cdot \sqrt{1 + x^2}\right)' = \left(\sqrt{x^2 + x^4}\right)' = \frac{1}{2\sqrt{x^2 + x^4}} \cdot (2x + 4x^3) = \frac{2x(1 + 2x^2)}{2x\sqrt{1 + x^2}} = \frac{1 + 2x^2}{\sqrt{1 + x^2}}
$$

---
3. $$y = \sqrt{x + \sqrt{x}}$$

$$
y' = \left(\sqrt{x + \sqrt{x}}\right)' = \frac{1}{2\sqrt{x + \sqrt{x}}} \cdot \left(1 + \frac{1}{2\sqrt{x}}\right) = \frac{1 + 2\sqrt{x}}{4\sqrt{x}\sqrt{x + \sqrt{x}}} = \frac{1 + 2\sqrt{x}}{4\sqrt{x^2 + x\sqrt{x}}}
$$

---
4. \* $$y = \sqrt{x + \sqrt{x + \sqrt{x}}}$$

---
5. $$y = (x^2 + 2)^5 \cdot (3x - x^3)^3$$

$$
y' = ((x^2 + 2)^5 \cdot (3x - x^3)^3)' = (x^2 + 2)^5 \cdot (3x - x^3)^3 \cdot \left( \frac{5}{x^2 + 2} \cdot 2x + \frac{3}{3x - x^3} \cdot (3-3x^2) \right) = (x^2 + 2)^5 \cdot (3x - x^3)^3 \cdot \left( \frac{10x}{x^2 + 2} + \frac{9 (1-x^2)}{3x - x^3}\right)
$$

---
6. \* $$y = (1 + x) \cdot (2 - x)^2 \cdot (x^2 - 3x)^3 \cdot \sin x$$

---
7. $$y = \sqrt[x]{x}$$

---
8. $$y = \frac{2x}{1 - x^2}$$

---
9. $$y = \frac{x}{(x + 1) \cdot (2 - 3x)}$$

---
10. \* $$y = \frac{(2 - x^2)^3 \cdot (x - 1)^2}{(2x^3 - 3x) \cdot e^x}$$

---
11. $$y = \ln\left( x + \sqrt{x^2 + 1}\right)$$

---
12. $$y = x \cdot \ln\left(x + \sqrt{x^2 + 1}\right) - \sqrt{x^2 + 1}$$

---
13. \* $$y = \arcsin(\sin x)$$

---
14. \* Найти длину 𝑥 и ширину 𝑦 прямоугольника при заданном периметре 𝑃 = 144 см, при которых данный прямоугольник имеет наибольшую площадь 𝑆.